In [90]:
path1 = 'C:\Users\leyang\Downloads\AAA_MDX\MDX4806-20160817081259226.TDS'
path2 = 'C:\Users\leyang\Downloads\AAA_MDX4806\MDX4806-20160817095942791.TDS'
gz1 = 'C:\Users\leyang\Downloads\AAA_MDX\SNK1551-20160602193817305.TDS.gz'

def transform_tds(gz_path):
    '''
    transform gz TDS file with desired format for Hive
    '''
    tags, svid = [], {}
    with gzip.open(gz_path, 'rb') as f:        
        with open(gz_path[0:-6]+'txt', 'w') as fw:
#             fw.write('%s\t%s\t%s\n' %('timestamp', 'context', 'data')) -- no need for hive
            for l in f.read().split('\n'):                
                elem = l.split(' ')
                # buffering tag
                if l.startswith('TAG'):                    
                    ctx = elem[1][6:elem[1].rfind('/')].replace('Eqp:','').replace('PA:','').replace('SPA:','')
                    svid[elem[2][5:]] = ctx
                    tags.append((l,ctx))
                # parsing value line
                if l.startswith('TD'):                    
                    ts = elem[1][3:].replace('-',' ').replace('/','-')
                    # write buffered tag line with timestamp and context
                    for t,ctx in tags:
                        fw.write('%s\t%s\t%s' %(ts, ctx, t))
                    tags = []
                    # write value line
                    sn = elem[2][4:-1].split('^')[0].split('=')[0]                
                    fw.write('%s\t%s\t%s' %(ts, svid[sn], l))
    print 'TDS transform completed!'

In [1]:
import gzip

gz1 = 'C:\Users\leyang\Downloads\AAA_MDX\SNK1551-20160602193817305.TDS.gz'
gz2 = 'C:\Users\leyang\Downloads\AAA_MDX\ETX4800-20160817091159382.TDS.gz'

def transform_tds_gz(gz_path):
    '''
    transform gz TDS file with desired format for Hive
    '''
    tags, svid, gz_out = [], {}, []
    with gzip.open(gz_path, 'rb') as f:        
        for l in f.read().split('\n'):                
            elem = l.split(' ')
            # buffering tag
            if l.startswith('TAG'):                    
                ctx = elem[1][6:elem[1].rfind('/')].replace('Eqp:','').replace('PA:','').replace('SPA:','').replace('/','_')
                svid[elem[-2][5:]] = ctx
                tags.append((l,ctx))
            # parsing value line
            if l.startswith('TD'):                    
                ts = elem[1][3:].replace('-',' ').replace('/','-')
                # write buffered tag line with timestamp and context
                for t,ctx in tags:
                    gz_out.append('%s\t%s\t%s' %(ts, ctx, t))
                tags = []
                # write value line
                sn = elem[2][4:-1].split('^')[0].split('=')[0]                
                gz_out.append('%s\t%s\t%s' %(ts, svid[sn], l))
            
    with gzip.open(gz_path[0:-6]+'gz', 'wb') as f:
        f.write('\n'.join(gz_out))
    print 'TDS transform completed for %s' %gz_path

In [78]:
d='2016/08/17-08:02:19.621	MDX4806/CHD	TAG NAME="Eqp:MDX4806/PA:CHD/BR2_LCF--rCfXCorrExt" SVID=21 UNITS="m"'
e='timestamp	context	data'
f='2016-08-17 08:02:19.621	MDX4806/CHD	TD TS=2016/08/17-08:02:19.621 SV={3=0^4=-1^5=29.741^6="HEAT"^7=3^8=0.656^9=309^10=16.5678884243294^11=-0.06^12=1800^13=8.054^14=3.27413814835256E-05^15=7.498779296875^16=3.736^17=-0.000367^18=-2.3E-05^19=-0.000397^20=-3.3E-05^21=0.000227^22=3.4E-05^23=-4.6E-05^24=0.000139}'
f.split('\t')

['2016-08-17 08:02:19.621',
 'MDX4806/CHD',
 'TD TS=2016/08/17-08:02:19.621 SV={3=0^4=-1^5=29.741^6="HEAT"^7=3^8=0.656^9=309^10=16.5678884243294^11=-0.06^12=1800^13=8.054^14=3.27413814835256E-05^15=7.498779296875^16=3.736^17=-0.000367^18=-2.3E-05^19=-0.000397^20=-3.3E-05^21=0.000227^22=3.4E-05^23=-4.6E-05^24=0.000139}']

In [2]:
import time
start_time = time.time()
transform_tds_gz(gz2)
print("--- %s seconds ---" % (time.time() - start_time))


TDS transform completed for C:\Users\leyang\Downloads\AAA_MDX\ETX4800-20160817091159382.TDS.gz
--- 1.21299982071 seconds ---


In [50]:
a='TD TS=2016/08/17-08:02:19.621 SV={3=0^4=-1^5=29.741^6="HEAT"^7=3^8=0.656^9=309^10=16.5678884243294^11=-0.06^12=1800^13=8.054^14=3.27413814835256E-05^15=7.498779296875^16=3.736^17=-0.000367^18=-2.3E-05^19=-0.000397^20=-3.3E-05^21=0.000227^22=3.4E-05^23=-4.6E-05^24=0.000139}'
b='TD TS=2016/08/17-08:02:19.851 SV={2="ProcessStarted"}'
b='TD TS=2016/08/17-08:02:19.861 SV={1=0}'
b.split(' ')[2][4:-1].split('^')[0].split('=')[0]

'1'

In [82]:
import gzip
with gzip.open('C:\Users\leyang\Downloads\AAA_MDX\SNK1551-20160602193817305.TDS.gz', 'rb') as f:
    file_content = f.read().split('\n')

In [57]:
import gzip, re
import numpy as np

sample_tag = 'TAG NAME="Eqp:SNK1551/PA:PM3L/ExhaustPressure02" SVID=58 UNITS=""'
sample_td = ' TD TS=2016/06/02-23:12:35.714 SV={191="SNK1551-20160602-0032"^192="UR62192.000"^193="rcp P-SC-50DHF-180TMAH-5L20P"^194="D-P-SNK155X. P-SC-50DHF-180TMAH-HS.01"^195="21"^196="PMGYB060MXA7"^198=1^201=1}'

p_tag = re.compile('TAG NAME="Eqp:(.*)" SVID=(\d*) UNITS="(.*)"')
p_td = re.compile('TD TS=(.*) SV={(.*)}')

gz1 = 'C:\Users\leyang\Downloads\AAA_MDX\SNK1551-20160602193817305.TDS.gz'

def check_tds_gz(gz_path):
    '''
    transform gz TDS file with desired format for Hive
    '''
    tags, svid, gz_out, cnt = [], {}, [], 0
    fid = gz_path[gz_path.index('-')+1:-7]
    with gzip.open(gz_path, 'rb') as f:        
        for l in f.read().split('\n'):   
            # buffering tag
            if l.startswith('TAG'):       
                p1, p2 = l.index('" SVID='), l.index(' UNITS="')
                tag, sid = l[10:p1], l[p1+7:p2]
                ctx = tag[:tag.rfind('/')].replace('Eqp:','').replace('PA:','').replace('SPA:','').replace('/','_')
                svid[sid] = ctx
                tags.append((l,ctx))
            # parsing value line
            if l.startswith('TD'):                    
                p1 = l.index(' SV=')
                timestamp, sv = l[6:p1], l[p1+1:]                
                ts = timestamp.replace('-',' ').replace('/','-')
                # write buffered tag line with timestamp and context
                for t,ctx in tags:
#                     print '%s\t%s\t%s\t%s' %(ts, ctx, fid, t)
                    gz_out.append('%s\t%s\t%s\t%s' %(ts, ctx, fid, t))
                tags = []
                # check if all values from the same chamber, write value line
                values = np.array(sv[4:-1].split('^'))
                sn = np.array([svid[kv.split('=')[0]] for kv in values])
                # split the TD row for individual chamber
                a_ctx = np.unique(sn)                
                if a_ctx.size > 1:
                    cnt += 1
#                     print (timestamp, a_ctx.size)
                    for ctx in a_ctx:
                        td = 'TD TS=%s SV={%s}' %(timestamp, '^'.join(values[sn==ctx]).strip('}'))
                        gz_out.append('%s\t%s\t%s\t%s' %(ts, ctx, fid, td))
                else:
                    gz_out.append('%s\t%s\t%s\t%s' %(ts, a_ctx[0], fid, l))
            
    with gzip.open(gz_path[0:-6]+'gz', 'wb') as f:
        f.write('\n'.join(gz_out))
    print 'TDS transform completed for "%s", with %d mixing TD lines.' %(gz_path, cnt)

In [56]:
gz2 = 'C:\GFApps\TDSParsing\AAA\ETX4800\ETX4800-20160817075058359.TDS.gz'
gz3 = 'C:\GFApps\TDSParsing\AAA\EPI1200\EPI1200-20160817090432846.TDS.gz'
gz4 = 'C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160602202022427.TDS.gz'
gz5 = 'C:\GFApps\TDSParsing\BBB\MDX4806\MDX4806-20160817080219812.TDS.gz'
check_tds_gz(gz5)

TDS transform completed for "C:\GFApps\TDSParsing\BBB\MDX4806\MDX4806-20160817080219812.TDS.gz", with 3205 mixing TD lines.


In [58]:
import os

folder = 'C:\GFApps\TDSParsing\AAA\SNK1551\\'

for f in os.listdir(folder):
    if not f.endswith('.TDS.gz'):
        continue
    check_tds_gz(folder + f)

TDS transform completed for "C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160602193817305.TDS.gz", with 1 mixing TD lines.
TDS transform completed for "C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160602195917496.TDS.gz", with 2 mixing TD lines.
TDS transform completed for "C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160602202022427.TDS.gz", with 2 mixing TD lines.
TDS transform completed for "C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160602204112409.TDS.gz", with 4 mixing TD lines.
TDS transform completed for "C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160602210222502.TDS.gz", with 5 mixing TD lines.
TDS transform completed for "C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160602212201047.TDS.gz", with 3 mixing TD lines.
TDS transform completed for "C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160602215613444.TDS.gz", with 2 mixing TD lines.
TDS transform completed for "C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160602221723473.TDS.gz", with 3 mixing TD lines.
TDS transform completed for "C:\

In [5]:
s = 'asdfsfaf'
s.endswith('af')

True

# Another way of parsing using regex

In [13]:
import gzip, re
import numpy as np

gz1 = 'C:\Users\leyang\Downloads\AAA_MDX\SNK1551-20160602193817305.TDS.gz'

def check_tds_gz(gz_path):
    '''
    transform gz TDS file with desired format for Hive
    '''
    
    p_tag = re.compile('TAG NAME="(.*)" SVID=(\d*) UNITS="(.*)"')
    p_td = re.compile('TD TS=(.*) SV={(.*)}')
    
    tags, svid, gz_out, cnt = [], {}, [], 0
    fid = gz_path[gz_path.index('-')+1:-7]
    with gzip.open(gz_path, 'rb') as f:        
        for l in f.read().split('\n'):   
            # buffering tag
            if l.startswith('TAG'):       
                tag, sid, unit = p_tag.match(l).groups()
                ctx = '_'.join([x.split(':')[1] for x in tag.split('/')[:-1]])
                svid[sid] = ctx
                tags.append((l, ctx))
            # parsing value line
            if l.startswith('TD'):    
                timestamp, sv = p_td.match(l).groups()
                ts = timestamp.replace('-',' ').replace('/','-')
                # write buffered tag line with timestamp and context
                for t,ctx in tags:
                    gz_out.append('%s\t%s\t%s\t%s' %(ts, ctx, fid, t))
                tags = []
                # check if all values from the same chamber, write value line
                values = np.array(sv.split('^'))
                sn = np.array([svid[kv.split('=')[0]] for kv in values])
                # split the TD row for individual chamber
                a_ctx = np.unique(sn)                
                if a_ctx.size > 1:
                    cnt += 1
#                     print (timestamp, a_ctx.size)
                    for ctx in a_ctx:
                        td = 'TD TS=%s SV={%s}' %(timestamp, '^'.join(values[sn==ctx]))
                        gz_out.append('%s\t%s\t%s\t%s' %(ts, ctx, fid, td))
                else:
                    gz_out.append('%s\t%s\t%s\t%s' %(ts, a_ctx[0], fid, l))
            
    with gzip.open(gz_path[0:-6]+'gz', 'wb') as f:
        f.write('\n'.join(gz_out))
    print 'Done for "%s", with %d mixing TD lines.' %(gz_path, cnt)

In [14]:
import os

folder = 'C:\GFApps\TDSParsing\AAA\EPI1200\821\\'

for f in os.listdir(folder):
    if not f.endswith('.TDS.gz'):
        continue
    check_tds_gz(folder + f)

Done for "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160821040152100.TDS.gz", with 100 mixing TD lines.
Done for "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160821042253779.TDS.gz", with 103 mixing TD lines.
Done for "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160821044354671.TDS.gz", with 106 mixing TD lines.
Done for "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160821050451740.TDS.gz", with 108 mixing TD lines.
Done for "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160821052552791.TDS.gz", with 108 mixing TD lines.
Done for "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160821054652203.TDS.gz", with 102 mixing TD lines.
Done for "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160821060753897.TDS.gz", with 113 mixing TD lines.
Done for "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160821062852949.TDS.gz", with 114 mixing TD lines.
Done for "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160821064957000.TDS.gz", with 106 mixing TD lines.
Done for "C:\GFApps\TDSParsi

# Hive result sanity check

In [105]:
import numpy as np
import re

aaa = '2016-08-17 08:43:30.681	TAG NAME="Eqp:EPI1200/PA:CHD/BottomPyrometerValue" SVID=447 UNITS=""	20160817090432846'

aaa.split('\t')

path = 'C:\GFApps\TDSParsing\AAA\q_test'

def afterCheck(path):
    '''
    '''
    p_tag = re.compile('TAG NAME="(.*)" SVID=(\d*) UNITS="(.*)"')
    p_td = re.compile('TD TS=(.*) SV={(.*)}') 
    tags = {}
    with open(path) as f:
        for l in f.readlines():
            ts, line, fid = l.split('\t')
            if line.startswith('TAG'):
                tag, svid, unit = p_tag.match(line).groups()
                tags[svid] = tag            
#                 print(tag,svid,unit)
            if line.startswith('TD'):
                ts, vals = p_td.match(line).groups()
                sid = [v.split('=')[0] in tags for v in vals.split('^')]
                if not np.all(sid):
                    print(l)
                    raise Exception('file corrupted: SVID not found')
#             break
    print('file %s is good' %path)

            
afterCheck(path)

file C:\GFApps\TDSParsing\AAA\q_test is good


In [159]:
def afterCheckCombine(path):
    '''
    '''
    p_tag = re.compile('TAG NAME="(.*)" SVID=(\d*) UNITS="(.*)"')
    p_td = re.compile('TD TS=(.*) SV={(.*)}') 
    tags, ts_val, last_ts, gz_out, tags_out = {}, '', None, [], {}
    with open(path) as f:
        for l in f.readlines():
            ts, line, fid = l.split('\t')
            if line.startswith('TAG'):
                tag, svid, unit = p_tag.match(line).groups()
                tags[svid] = tag         
                tags_out[int(svid)] = line
#                 print(tag,svid,unit)
            
            if line.startswith('TD'):
                # write tag into gz_out buffer, if any
                if tags_out:
                    gz_out += [tags_out[x] for x in sorted(tags_out)]
                    tags_out = {}
                    
                # sanity check for current timestamp
                ts, vals = p_td.match(line).groups()
                sid = [v.split('=')[0] in tags for v in vals.split('^')]
                if not np.all(sid):
                    print(l)
                    raise Exception('file corrupted: SVID not found')
                
                # write TD line with previous timestamp
                if ts_val and ts!=last_ts:
                    gz_out.append('TD TS=%s SV={%s}' %(last_ts, ts_val.strip('^')))
                    ts_val = vals
                else:
                    ts_val += ('^' + vals)
                
                # update timestamp before move to next line
                last_ts = ts
                
        # last row
        if ts_val:
            gz_out.append('TD TS=%s SV={%s}' %(last_ts, ts_val.strip('^')))
                    
    with gzip.open(path+'_check.gz', 'wb') as f:
        f.write('\n'.join(gz_out))

    print('file %s is good, and sorted gz is generated' %path)
    
path = 'C:\GFApps\TDSParsing\AAA\q_test_1'
afterCheckCombine(path)

file C:\GFApps\TDSParsing\AAA\q_test is good, and sorted gz is generated


# Another Hive query with collect_set

bin/hive -e "select ts, **collect_set**(data) as ts_data from tds_parsing.trace where context='EPI1200_CHD' and day in ('2016-08-17') and fid in ('20160817090432846','20160817092432121') and ((ts>='2016-08-17 09:03:21.909' and ts<='2016-08-17 09:21:14.7') or data like 'TAG%') group by ts order by ts;" > q_test

In [228]:
def postCheck(path):
    '''
    '''
    gz_out, svid = [], []
    p_td = re.compile('"TD TS=(.*) SV={(.*)}"')
    p_tag = re.compile('TAG NAME="(.*)" SVID=(\d*) UNITS="(.*)')
    with open(path) as f:
        for l in f.readlines():
            tsd = np.array(l.split('\t')[1].strip()[1:-1].split(','))
            
            # check tag
            isTag = np.array([x.startswith('"TAG') for x in tsd])
            if np.any(isTag):                     
                newTag = [x[1:-1].replace('\\','') for x in tsd[isTag]]
                gz_out += newTag
                svid += [p_tag.match(x).groups()[1] for x in newTag]
                
            # check td
            isTd = ~isTag
            if np.any(isTd):
                svs = tsd[isTd]
                val = [p_td.match(x).groups()[1].replace('\\','') for x in svs]
                # check if svid number is defined
                if not np.all([np.all([x.split('=')[0] in svid for x in v.split('^')]) for v in val]):
                    print(l)
                    raise Exception('file is corrupted, svid definition not found')
                # insert in gz out                
                gz_out.append('TD TS=%s SV={%s}' %(p_td.match(svs[0]).groups()[0], '^'.join(val)))
    
    while gz_out[-1].startswith('TAG'):
        gz_out = gz_out[:-1]
    
    with gzip.open(path + '_check.gz', 'wb') as f:
        f.write('\n'.join(gz_out))
        
    print('file %s is good, and sorted gz is generated' %path)
    
path = 'C:\GFApps\TDSParsing\AAA\q_test_2'

import time
start_time = time.time()
postCheck(path)
print("--- %s seconds ---" % (time.time() - start_time))


file C:\GFApps\TDSParsing\AAA\q_test_2 is good, and sorted gz is generated
--- 2.1850001812 seconds ---


# HBase Test

In [1]:
# https://github.com/eleme/thriftpy/issues/234
import happybase

In [66]:
# hbase thrift start -threadpool
connection = happybase.Connection('fc8xsiteg04', 9090)
connection.open()

In [67]:
connection.tables()

['test']

In [68]:
table = connection.table('test')

In [58]:
table.row(b'row2', columns=[b'cf:b'])

{'cf:b': 'value2'}

In [59]:
table.put(b'row4', {b'cf2:c1': b'value1', b'cf2:c2': b'value2'})

In [60]:
table.row(b'row4')

{'cf2:c1': 'value1', 'cf2:c2': 'value2', 'cf:c1': 'value1', 'cf:c2': 'value2'}

In [69]:
table.families()

{'cf': {'block_cache_enabled': True,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': 'ROW',
  'bloom_filter_vector_size': 0,
  'compression': 'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': 'cf:',
  'time_to_live': 2147483647},
 'cf2': {'block_cache_enabled': True,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': 'ROW',
  'bloom_filter_vector_size': 0,
  'compression': 'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': 'cf2:',
  'time_to_live': 2147483647}}

# HBase Insert with REST API

In [209]:
import gzip, re, time
import numpy as np
from starbase import Connection
from datetime import datetime

gz1 = "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160822033914352.TDS.gz"

def tds_hbase_insert(gz_path):
    '''
    read raw TDS file from ETC, and insert data into HBase
    '''

    # hbase connection
    # hbase rest start
    c = Connection(host='fc8xsiteg04', port=8080)
    etc_table = c.table(gz_path.split('\\')[-1].split('-')[0])
    if not etc_table.exists():
        etc_table.create('ETC_Stats')
        print 'created new table %s' %etc_table.name
        
    epoch = datetime.utcfromtimestamp(0)
    p_tag = re.compile('TAG NAME="(.*)" SVID=(\d*) UNITS="(.*)"')
    p_td = re.compile('TD TS=(.*) SV={(.*)}')
    
    svid, n_svid, batch = {}, 0, {}
    i, tot = 1, 0
    with gzip.open(gz_path, 'rb') as f:        
        for l in f.read().split('\n'):   
            # buffering tag
            if l.startswith('TAG'):       
                tag, sid, unit = p_tag.match(l).groups()
                tag_elem = tag.split('/')
                ctx = '_'.join([x.split(':')[1] for x in tag_elem[1 if len(tag_elem)>2 else 0:-1]]) 
                svid[sid] = {'context':ctx, 'sensor':tag_elem[-1], 'etc':tag_elem[0].split(':')[1]}
                                
            # parsing value line
            if l.startswith('TD'):    
                timestamp, sv = p_td.match(l).groups()
                timestamp = datetime.strptime(timestamp, '%Y/%m/%d-%H:%M:%S.%f')
                timestamp = int((timestamp - epoch).total_seconds() * 1000)
                if timestamp not in batch:
                    batch[timestamp] = {}
                for x in svid.values():
                    if x['context'] not in batch[timestamp]:
                        batch[timestamp][x['context']] = {}
                i += 1        
                # check if all values from the same chamber, write value line
                for val in sv.split('^'):
                    s,v = val.split('=')                    
                    batch[timestamp][svid[s]['context']][svid[s]['sensor']] = v.strip('"')
                
            # batch insert
            if i%20000==0:
#                 print(i,n_svid,len(svid))
                # add new columns                 
                if len(svid) > n_svid:
                    cf = etc_table.columns()
                    ctx = np.unique([x['context'] for x in svid.values()])                    
                    newCol = [col for col in ctx if col not in cf]
                    if newCol:                        
                        cmd = "etc_table.add_columns('%s')" %("','".join(newCol))
                        exec(cmd)
                        time.sleep(10)
                        print 'added new column family: %s' %str(newCol)
                    
                    n_svid = len(svid)  
                    
                # insert data
                b = etc_table.batch()
                if b:
                    for row, data in batch.items():
                        b.insert(str(row), data)
                    b.commit(finalize=True)                                      
                print 'batch inserted %d rows!' %len(batch)
                tot += len(batch)
                batch = {} 
#                 time.sleep(30)
        
        # insert what's left
        if batch:
            b = etc_table.batch()
            if b:
                for row, data in batch.items():
                    b.insert(str(row), data)
                b.commit(finalize=True)                                                      
            
    tot += len(batch)
    print 'Done inserting for "%s" with total %d rows.' %(gz_path, tot)



In [210]:
gz = "C:\GFApps\TDSParsing\AAA\SNK1551\SNK1551-20160603002414090.TDS.gz"
gz = "C:\GFApps\TDSParsing\AAA\EPI1200\EPI1200-20160817092432121.TDS.gz"
tds_hbase_insert(gz)

batch inserted 788 rows!
batch inserted 740 rows!
batch inserted 811 rows!
batch inserted 813 rows!
added new column family: ['SWLLA_SWLLA8']
batch inserted 758 rows!
added new column family: ['SWLLA_SWLLA6']
batch inserted 849 rows!
batch inserted 750 rows!
batch inserted 782 rows!
batch inserted 808 rows!
batch inserted 791 rows!
batch inserted 698 rows!
batch inserted 881 rows!
added new column family: ['SWLLA_SWLLA9']
batch inserted 776 rows!
added new column family: ['SWLLA_SWLLA7']
batch inserted 806 rows!
Done inserting for "C:\GFApps\TDSParsing\AAA\EPI1200\EPI1200-20160817092432121.TDS.gz" with total 11159 rows.


# HBase insert with Thrift

In [44]:
import gzip, re, time, happybase, starbase
import numpy as np
from datetime import datetime

gz1 = "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160822033914352.TDS.gz"

def add_column(etc, newCol):
    '''
    '''
    
    table = starbase.Connection('fc8xsiteg03', port=8080).table(etc)    
    if newCol:                        
        cmd = "table.add_columns('%s')" %("','".join(newCol))
        exec(cmd)
        time.sleep(5)
        print 'added new column family: %s' %str(newCol)

def tds_hbase_insert_thrift(gz_path):
    '''
    read raw TDS file from ETC, and insert data into HBase
    '''

    # hbase connection
    # hbase thrift start -threadpool
    c = happybase.Connection(host='fc8xsiteg04', port=9090)    
    etc = gz_path.split('\\')[-1].split('-')[0]
    etc_table = c.table(etc)
    if etc not in c.tables():
        c.create_table(etc, {'ETC_Stats':dict(max_versions=1)})       
        print 'created new table %s' %etc_table.name
        
    epoch = datetime.utcfromtimestamp(0)
    p_tag = re.compile('TAG NAME="(.*)" SVID=(\d*) UNITS="(.*)"')
    p_td = re.compile('TD TS=(.*) SV={(.*)}')
    
    svid, n_svid, batch = {}, 0, {}
    i, tot = 1, 0
    with gzip.open(gz_path, 'rb') as f:        
        for l in f.read().split('\n'):   
            # buffering tag
            if l.startswith('TAG'):       
                tag, sid, unit = p_tag.match(l).groups()
                tag_elem = tag.split('/')
                ctx = '_'.join([x.split(':')[1] for x in tag_elem[1 if len(tag_elem)>2 else 0:-1]]) 
                svid[sid] = {'context':ctx, 'sensor':tag_elem[-1], 'etc':tag_elem[0].split(':')[1]}
                                
            # parsing value line
            if l.startswith('TD'):    
                i += 1
                timestamp, sv = p_td.match(l).groups()
                timestamp = datetime.strptime(timestamp, '%Y/%m/%d-%H:%M:%S.%f')
                timestamp = int((timestamp - epoch).total_seconds() * 1000)
                if timestamp not in batch:
                    batch[timestamp] = {}                
                # check if all values from the same chamber, write value line
                for val in sv.split('^'):
                    s,v = val.split('=')                    
                    batch[timestamp][(svid[s]['context']+':'+svid[s]['sensor']).encode()] = v.strip('"').encode()
                
            # batch insert
            if i%15000==0:
                # add new columns                 
                if len(svid) > n_svid:
                    ctx = np.unique([x['context'] for x in svid.values()])
                    cf = etc_table.families().keys()                    
                    newCol = [f for f in ctx if f not in cf]
                    if newCol:
                        add_column(etc, newCol)
                        c = happybase.Connection(host='fc8xsiteg04', port=9090)  
                        etc_table = c.table(etc)
                    n_svid = len(svid)  
                    
                # insert data
                b = etc_table.batch()
                for row, data in batch.items():
                    b.put(str(row).encode(), data)
                b.send()                                  
                print 'batch inserted %d rows!' %len(batch)
                tot += len(batch)
                batch = {} 
#                 time.sleep(30)
        
        # insert what's left
        if batch:
            b = etc_table.batch()            
            for row, data in batch.items():
                b.put(str(row).encode(), data)
            b.send()                                               
            
    tot += len(batch)
    c.close()
    print 'Done inserting for "%s" with total %d rows.' %(gz_path, tot)


In [45]:
gz = "C:\GFApps\TDSParsing\AAA\EPI1200\821\EPI1200-20160822013313558.TDS.gz" #52100
tds_hbase_insert_thrift(gz)

batch inserted 562 rows!
batch inserted 626 rows!
batch inserted 566 rows!
batch inserted 554 rows!
batch inserted 638 rows!
batch inserted 576 rows!
batch inserted 587 rows!
batch inserted 640 rows!
batch inserted 572 rows!
batch inserted 615 rows!
batch inserted 573 rows!
batch inserted 565 rows!
batch inserted 617 rows!
batch inserted 563 rows!
batch inserted 627 rows!
batch inserted 552 rows!
batch inserted 551 rows!
batch inserted 624 rows!


timeout: timed out

In [36]:
c=happybase.Connection('fc8xsiteg04', port=9090)
tbl = c.table('SNK1551')
tbl.families()
# c.

{'ETC_Stats': {'block_cache_enabled': True,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': 'ROW',
  'bloom_filter_vector_size': 0,
  'compression': 'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': 'ETC_Stats:',
  'time_to_live': 2147483647},
 'PM1L': {'block_cache_enabled': True,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': 'ROW',
  'bloom_filter_vector_size': 0,
  'compression': 'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': 'PM1L:',
  'time_to_live': 2147483647},
 'PM1R': {'block_cache_enabled': True,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': 'ROW',
  'bloom_filter_vector_size': 0,
  'compression': 'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': 'PM1R:',
  'time_to_live': 2147483647},
 'PM2L': {'block_cache_enabled': True,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': 'ROW',
  'bloom_filter_vector_size': 0,
  'compression': 'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': 'PM2L:',
  'time_to_live':

In [194]:
type(aaa.encode())

str